# **CH01.1. Skip-gram & CBOW**

#### **`Paper Info`** : Efficient Estimation of Word Representations in Vector Space
#### $ \hspace{1.75cm} - $ Tomas Mikolov, Kai Chen, Greg Corrado, Jeffrey Dean et al. (ICLR/2013)

<b></b>

> ## **요약(Summary)**

| Item | Description |
|------|------|
| Research topic | 대규모 corpus 환경에서 효율적으로 단어 분산 표현을 학습하는 log-linear 기반 방법 제안 |
| Core idea | 비선형 은닉층 없이 얕은 구조로 CBOW·Skip-gram을 제안하고, 출력 근사를 Hierarchical Softmax 또는 Negative Sampling으로 처리하며, <br> 빈도 상위 단어 Subsampling으로 학습 효율을 개선 |
| Key findings | $ \cdot{} $ NNLM/RNNLM 대비 예제당 연산량 대폭 감소(softmax 근사로 출력 비용 절감) <br> $ \cdot{} $ 대규모 데이터에서도 단일 머신에서 빠른 학습이 가능함을 보임 <br> $ \cdot{} $ 임베딩이 의미적 유사성을 정성적으로 반영함을 관찰 <br> $ \cdot{} $ 빈도 상위 단어 Subsampling으로 학습 시간 단축 및 임베딩 품질 개선 |
| Contributions | $ \cdot{} $ 얕은 log-linear 구조 제시 <br> $ \cdot{} $ Hierarchical Softmax로 출력 비용 $ O \big({} \log{}_{2}(V) \big){} $ 달성 <br> $ \cdot{} $ 대규모 corpus에서의 학습 효율성 실증 <br> $ \cdot{} $ 빈도 기반 통계적 특징에 의존하지 않고 의미적 유사성을 학습하는 연속 표현의 실용성 입증 <br> $ \cdot{} $ Negative Sampling으로 대규모 어휘 출력 계산을 간단하고 효율적으로 근사 <br> $ \cdot{} $ 빈도 기반 Subsampling으로 자주 등장하는 불용성 단어의 영향 완화 |

<b></b>

> ## **연구 배경(Motivation & Prior Work)**

#### **(1) 기존 연구 및 한계점** :
##### $ \hspace{0.15cm} \cdot{} $ 기존 NNLM(Neural Network Language Model)은 $ p(w_{t} \mid{} w_{t-n+1}, \cdots{}, w_{t-1}) $를 모델링하지만, 은닉층의 계산복잡도가 높음
##### $ \hspace{0.15cm} \cdot{} $ RNNLM(RNN-Language Model)은 장기 의존성은 포착하지만, $ O(H^{2} + H V) $로 학습 속도가 느림
##### $ \hspace{0.15cm} \cdot{} $ 수십억 토큰 규모의 대규모 데이터셋에서 이러한 복잡도가 학습 병목으로 작용
##### $ \hspace{0.15cm} \cdot{} $ SVD 기반 LSA(Latent Semantic Analysis) 등의 통계적 방법은 희소 행렬 분해에 의존해 의미적 일반화 능력이 제한됨

#### **(2) 연구 목표** :
##### $ \hspace{0.15cm} \cdot{} $ 은닉층을 제거한 얕은 log-linear 구조로 효율적인 분포 표현 학습 수행
##### $ \hspace{0.15cm} \cdot{} $ 의미적·통사적 관계를 반영하는 고품질 단어 임베딩 학습
##### $ \hspace{0.15cm} \cdot{} $ 확률적 예측 구조를 유지하면서 연산 효율 향상

#### **(3) 제안된 방법론** :
##### $ \hspace{0.15cm} \cdot{} $ CBOW model을 통해 주변 단어의 임베딩 평균으로 중심 단어를 예측해 맥락 기반의 단어 의미를 효율적으로 학습하고 학습 복잡도를 감소시킴
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram model을 통해 중심 단어로 주변 단어를 예측해 희귀 단어의 표현력을 강화하고 의미적·통사적 관계를 보다 정교하게 포착함

<b></b>

> ## **방법론(Method)**

#### **(1) Skip-gram** :
##### $ \hspace{0.15cm} \cdot{} $ 중심 단어 $ w_{t} $로부터 주변 단어 $ w_{t+j} $를 예측을 목표
##### $ \hspace{0.15cm} \cdot{} $ Loss function은 Negative log-likelihood로 정의
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} \mathcal{L}_{\text{SG}}(\theta{}) = - \sum_{t=1}^{T} \sum_{\substack{-c \le j \le c \\ j \neq{} 0}} \log{}\big({} p(w_{t+j} \mid{} w_{t}; \theta{})\big){} $
##### $ \hspace{0.45cm} \text{where } \, w_{t} : \text{input(center word) index(scalar)} $
##### $ \hspace{0.45cm} \text{and } \, w_{t+j} : \text{output(predicted word) index} $
##### $ \hspace{0.15cm} \cdot{} $ 조건부 확률 $ p(w_{t+j} \mid{} w_{t}; \theta{}) $은 실제 hierarchical softmax로 계산하여 근사함
##### $ \hspace{0.15cm} \Rightarrow{} p(w_{o} \mid{} w_{i}) = \frac{\exp{}\big(\textbf{u}_{w_{o}}^{\top} \textbf{v}_{w_{i}}\big)}{\sum_{w' \in{} \mathcal{V}} \exp{}\big(\textbf{u}_{w'}^{\top} \textbf{v}_{w_{i}}\big)} $
##### $ \hspace{0.45cm} \text{where } \, \textbf{v}_{w_{i}} : \text{input(center word) embedding vector} $
##### $ \hspace{0.45cm} \text{and } \, \textbf{u}_{w_{o}} : \text{output(predicted word) embedding vector} $
##### $ \hspace{0.45cm} \text{and } \, \textbf{v}_{w} : \text{embedding used as the final word representation after training} $

#### **(2) CBOW(Continuous Bag-of-Words)** :
##### $ \hspace{0.15cm} \cdot{} $ 목표 : 주변 단어 집합 $ \mathcal{C} = \{ w_{t-c}, \cdots{}, w_{t-1}, w_{t+1}, \cdots{}, w_{t+c} \} $으로부터 중심 단어 $ w_{t} $를 예측
##### $ \hspace{0.15cm} \cdot{} $ Loss function은 Negative log-likelihood로 정의
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} \mathcal{L}_{\text{CBOW}}(\theta{}) = - \sum_{t=1}^{T} \log{} p(w_{t} | \mathcal{C}; \theta{}) $
##### $ \hspace{0.45cm} \text{where } \, w_{t} : \text{output(center word) index} $
##### $ \hspace{0.45cm} \text{and } \, \mathcal{C} : \text{input(context word) indices within window} $
##### $ \hspace{0.15cm} \cdot{} $ 주변 단어 임베딩의 평균을 통해 맥락 벡터 $ \bar{\textbf{v}}_{t} $를 구성
##### $ \hspace{0.15cm} \Rightarrow{} \bar{\textbf{v}}_{t} = \displaystyle{} \frac{1}{2c} \sum_{\substack{-c \le j \le c \\ j \neq 0}} \textbf{v}_{w_{t+j}} $
##### $ \hspace{0.15cm} \cdot{} $ 조건부 확률 $ p(w_{t} | \bar{\textbf{v}}_{t}; \theta{}) $은 softmax 또는 hierarchical softmax로 근사됨
##### $ \hspace{0.15cm} \Rightarrow{} p(w_{t} | \bar{\textbf{v}}_{t}) = \frac{\exp{}\big(\textbf{u}_{w_{t}}^{\top} \bar{\textbf{v}}_{t}\big)}{\sum_{w' \in \mathcal{V}} \exp{}\big(\textbf{u}_{w'}^{\top} \bar{\textbf{v}}_{t}\big)} $
##### $ \hspace{0.45cm} \text{where } \, \textbf{v}_{w_{t+j}} : \text{input(context word) embedding vectors} $
##### $ \hspace{0.45cm} \text{and } \, \textbf{u}_{w_{t}} : \text{output(center word) embedding vector} $
##### $ \hspace{0.45cm} \text{and } \, \textbf{v}_{w} : \text{embedding used as the final word representation after training} $

##### **(`PLUS`) 계층적 소프트맥스(Hierarchical Softmax)** :
##### $ \hspace{0.15cm} \cdot{} $ 각 단어를 리프로 하는 이진 트리에서 내부 노드 $ n $ 마다 매개변수 $ \mathbf{u}_{n} \in{} \mathbb{R}^{D} $ 를 두고 경로를 따라 이진 분류 확률을 곱함
##### $ \hspace{0.15cm} \Rightarrow{} p\{ w \mid \mathbf{x} \} = \prod_{n \in{} \text{path}(w)} \sigma\{ s(n,w)\, \mathbf{u}_{n}^{\top} \mathbf{x} \} \;\; \text{ where } \, s(n,w) \in{} \{ -1,+1 \} $
##### $ \hspace{0.15cm} \cdot{} $ 허프만 코딩으로 빈도 높은 단어의 경로를 짧게 하여 기대 연산량을 추가 절감

##### **(`PLUS`) 계산 복잡도(Computational Complexity;$ Q $)** :
##### $ \hspace{0.15cm} \cdot{} \, Q_{\text{NNLM}} = N D + N D H + H V \;\; \cdots{} \, (\text{baseline}) $ 
##### $ \hspace{0.15cm} \cdot{} \, Q_{\text{RNNLM}} = H^{2} + H V \;\; \cdots{} \, (\text{baseline}) $ 
##### $ \hspace{0.15cm} \cdot{} \, Q_{\text{CBOW}} = N D + D \log_{2}{V} $ 
##### $ \hspace{0.15cm} \cdot{} \, Q_{\text{SG}} = C \cdot(D + D \log_{2}{V}) $ 
##### $ \hspace{0.3cm} \text{where } \, N : \text{number of context words within a window(context size)} $ 
##### $ \hspace{0.3cm} \text{and } \, D : \text{embedding dimension(vector size)} $ 
##### $ \hspace{0.3cm} \text{and } \, H : \text{hidden layer dimension} $ 
##### $ \hspace{0.3cm} \text{and } \, V : \text{vocabulary size(number of unique words)} $ 
##### $ \hspace{0.3cm} \text{and } \, C : \text{maximum distance between input and predicted words (window radius)} $

<b></b>

> ## **실험(Experiments)**

#### **(1) 실험 설정** :
##### $ \hspace{0.15cm} \cdot{} $ 데이터셋
| Item | Description |
|------|--------------|
| Training dataset | Large news corpus ($ 6B $ tokens) |
| Data(Sentence;row) size | ? |
| Vocabulary(token) | $ 1M $ most frequent words |
| Evaluation dataset | None (qualitative observation only) |
| Evaluation size | Large-scale analogy benchmark introduced in later work |
##### $ \hspace{0.15cm} \cdot{} $ 모델 및 훈련
| Item | Description |
|------|--------------|
| Model type | Continuous Bag-of-Words (CBOW) / Skip-gram |
| Hidden layer | None (log-linear projection structure) |
| Embedding dimension | $ 50 / 100 / 300 $ |
| Context window size | Bidirectional window (typically $ c = 5 $) |
| Optimizer | Stochastic Gradient Descent(SGD) |
| Learning rate | Initial $ 0.025 $, linearly decayed |
| Training hardware | Single-machine CPU implementation (multi-threaded) |
| Training time | $ \approx{} 1 $ day for $ 6B $ tokens at $ D = 300 $ |
| Baseline models | Feed-forward NNLM, RNNLM |
| Evaluation metric | Training efficiency and computational cost comparison |

#### **(2) 분석 및 결과** :
##### $ \hspace{0.15cm} \cdot{} $ 차원 $ D $와 데이터 크기를 함께 증가시킬 때 품질 향상이 가장 큼
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram은 희소 단어와 더 넓은 맥락 포착에 유리하도록 설계되고 CBOW는 평균화 기반으로 빠르고 안정적인 학습을 제공
##### $ \hspace{0.15cm} \cdot{} $ NNLM/RNNLM 대비 같은 시간 내 더 많은 토큰을 처리함

#### **(3) 결과 해석** :
##### $ \hspace{0.15cm} \cdot{} $ CBOW는 평균 맥락 표현으로 계산을 단순화하여 효율을 확보
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram은 중심 단어로 여러 주변 단어를 예측해 희소 패턴 학습에 유리
##### $ \hspace{0.15cm} \cdot{} $ Hierarchical Softmax는 $ O\{ \log_{2}{V} \} $ 로 출력 정규화 병목을 완화
##### $ \hspace{0.15cm} \cdot{} $ 아날로지 대규모 정확도와 구체적 벡터 산술 평가는 후속 논문에서 집중적으로 다룸

<b></b>

> ## **결론(Conclusion)**

#### **(1)** CBOW와 Skip-gram은 단순 구조로 대규모 데이터 학습에 실용적임을 보임

#### **(2)** Hierarchical Softmax 도입으로 대어휘에서의 계산 병목을 완화

<b></b>

> ## **부록(Appendix)**

#### - 추가 수식 유도, 증명 과정
#### - Supplementary 실험 결과 (e.g. ablation table)
#### - 논문 원문 대비 수정된 notation, 논문 내 오타 정리

<b></b>

> ## **비평(Commentary)**

#### - 논문에 대한 비판적 의견 또는 보완 아이디어
#### - 수식적/모델적 한계 및 개선 방향 제안
#### - 구현 난이도나 재현성 관련 소감
#### - 후속 연구로의 확장 아이디어 제시

<b></b>

> ## **참고 문헌(Reference)**

#### **(1) 관련 후속 논문** :
##### $ \hspace{0.15cm} \cdot{} $ 

#### **(2) 기타 참고 자료** :
#### - 블로그, 연구실 발표 자료 등등